# Lab-04-1 Multivariate Linear regression

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim

## Data

In [2]:
x_train = torch.FloatTensor([[73, 80, 75], 
                             [93, 88, 93], 
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## Model without torch.nn

### Model Initialization

In [3]:
W = torch.randn(x_train.size(1), requires_grad=True)
b = torch.randn(1, requires_grad=True)
W, b

(tensor([ 0.6616, -0.1479, -0.3434], requires_grad=True),
 tensor([0.5755], requires_grad=True))

### Optimizer

In [4]:
optimizer = optim.SGD([W, b], lr=1e-5)

### Predict

In [5]:
x_train.matmul(W) + b

tensor([11.2906, 17.1589, 15.0991, 15.2616, 15.0773], grad_fn=<AddBackward0>)

### Train

In [6]:
n_epochs = 20
for epoch in range(n_epochs + 1):
    
    output = (x_train.matmul(W) + b).unsqueeze(1)
    
    loss = torch.mean((output - y_train)**2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))
    print('W: {} b: {:.2f}'.format(W.detach().numpy(), b.item()))

Epoch    0/20 y_pred: tensor([11.2906, 17.1589, 15.0991, 15.2616, 15.0773]) Cost: 24787.566406
W: [ 0.9302702   0.12062967 -0.0715034 ] b: 0.58
Epoch    1/20 y_pred: tensor([72.7760, 91.0593, 87.9147, 94.5559, 71.4446]) Cost: 7769.993164
W: [1.0806886  0.2709602  0.08069359] b: 0.58
Epoch    2/20 y_pred: tensor([107.1995, 132.4334, 128.6815, 138.9499, 103.0025]) Cost: 2435.887451
W: [1.164889   0.35513872 0.16590251] b: 0.58
Epoch    3/20 y_pred: tensor([126.4720, 155.5972, 151.5053, 163.8045, 120.6706]) Cost: 763.928772
W: [1.2120163  0.40228114 0.21360727] b: 0.58
Epoch    4/20 y_pred: tensor([137.2621, 168.5656, 164.2836, 177.7197, 130.5621]) Cost: 239.858368
W: [1.238388   0.42868838 0.24031483] b: 0.58
Epoch    5/20 y_pred: tensor([143.3032, 175.8261, 171.4377, 185.5104, 136.1000]) Cost: 75.589951
W: [1.2531391  0.44348678 0.25526685] b: 0.58
Epoch    6/20 y_pred: tensor([146.6855, 179.8910, 175.4431, 189.8721, 139.2003]) Cost: 24.100239
W: [1.2613844  0.45178577 0.26363733] b: 0.

## Manual Model

In [9]:
import torch
from torch import optim
import torch.nn as nn

In [10]:
class ManualLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.W = nn.Parameter(torch.randn(x_train.size(1), requires_grad=True))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True))
    
    def forward(self, x):
        return (x.matmul(self.W) + self.b).unsqueeze(1)

In [11]:
model = ManualLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [12]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([110.0358, 115.1027, 122.5886, 132.4487,  83.8398]) Cost: 3472.813232
Epoch    1/20 y_pred: tensor([132.8509, 142.5303, 149.6106, 161.8753, 104.7613]) Cost: 1129.018311
Epoch    2/20 y_pred: tensor([145.6229, 157.8870, 164.7388, 178.3498, 116.4757]) Cost: 394.340820
Epoch    3/20 y_pred: tensor([152.7722, 166.4855, 173.2082, 187.5730, 123.0353]) Cost: 164.037415
Epoch    4/20 y_pred: tensor([156.7735, 171.3004, 177.9494, 192.7365, 126.7090]) Cost: 91.827576
Epoch    5/20 y_pred: tensor([159.0124, 173.9969, 180.6035, 195.6270, 128.7669]) Cost: 69.171944
Epoch    6/20 y_pred: tensor([160.2646, 175.5075, 182.0891, 197.2450, 129.9203]) Cost: 62.048866
Epoch    7/20 y_pred: tensor([160.9643, 176.3541, 182.9203, 198.1505, 130.5672]) Cost: 59.794605
Epoch    8/20 y_pred: tensor([161.3548, 176.8290, 183.3853, 198.6572, 130.9305]) Cost: 59.066334
Epoch    9/20 y_pred: tensor([161.5721, 177.0958, 183.6453, 198.9406, 131.1352]) Cost: 58.816387
Epoch   10/20 y_pred: te

## Nested Models

In [15]:
import torch
from torch import optim
import torch.nn as nn

In [16]:
class LayerLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 1)
    
    def forward(self, x):
        return self.linear(x)

In [17]:
model = LayerLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [18]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([10.4816,  5.9370,  9.3333,  9.6368,  3.2908]) Cost: 27037.925781
Epoch    1/20 y_pred: tensor([74.6810, 83.1025, 85.3645, 92.4330, 62.1493]) Cost: 8483.892578
Epoch    2/20 y_pred: tensor([110.6233, 126.3051, 127.9314, 138.7874,  95.1025]) Cost: 2668.183105
Epoch    3/20 y_pred: tensor([130.7455, 150.4931, 151.7628, 164.7394, 113.5523]) Cost: 845.262878
Epoch    4/20 y_pred: tensor([142.0106, 164.0354, 165.1050, 179.2688, 123.8823]) Cost: 273.869476
Epoch    5/20 y_pred: tensor([148.3168, 171.6177, 172.5746, 187.4032, 129.6662]) Cost: 94.763107
Epoch    6/20 y_pred: tensor([151.8469, 175.8632, 176.7563, 191.9572, 132.9050]) Cost: 38.618080
Epoch    7/20 y_pred: tensor([153.8227, 178.2405, 179.0974, 194.5066, 134.7188]) Cost: 21.014811
Epoch    8/20 y_pred: tensor([154.9282, 179.5719, 180.4079, 195.9338, 135.7348]) Cost: 15.492414
Epoch    9/20 y_pred: tensor([155.5465, 180.3177, 181.1414, 196.7327, 136.3042]) Cost: 13.756691
Epoch   10/20 y_pred: tensor([1

### use torch.nn.functional as F

In [19]:
import torch.nn.functional as F

In [20]:
class LayerLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 1)
    
    def forward(self, x):
        return self.linear(x)

In [21]:
model = LayerLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [22]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = F.mse_loss(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([-50.1875, -55.0341, -56.9127, -62.6403, -40.0790]) Cost: 50934.269531
Epoch    1/20 y_pred: tensor([37.9498, 50.8997, 47.4660, 51.0255, 40.7216]) Cost: 15966.099609
Epoch    2/20 y_pred: tensor([ 87.2949, 110.2080, 105.9039, 114.6629,  85.9587]) Cost: 5005.436523
Epoch    3/20 y_pred: tensor([114.9215, 143.4124, 138.6212, 150.2911, 111.2852]) Cost: 1569.849854
Epoch    4/20 y_pred: tensor([130.3889, 162.0022, 156.9384, 170.2381, 125.4643]) Cost: 492.975159
Epoch    5/20 y_pred: tensor([139.0487, 172.4098, 167.1937, 181.4058, 133.4026]) Cost: 155.431839
Epoch    6/20 y_pred: tensor([143.8972, 178.2365, 172.9352, 187.6582, 137.8467]) Cost: 49.629513
Epoch    7/20 y_pred: tensor([146.6118, 181.4986, 176.1498, 191.1587, 140.3347]) Cost: 16.465590
Epoch    8/20 y_pred: tensor([148.1319, 183.3248, 177.9496, 193.1185, 141.7274]) Cost: 6.070072
Epoch    9/20 y_pred: tensor([148.9830, 184.3470, 178.9572, 194.2158, 142.5070]) Cost: 2.811227
Epoch   10/20 y_pred: ten

## Sequential Model

In [23]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

In [24]:
model = nn.Sequential(nn.Linear(x_train.size(1), 1))
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [25]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = F.mse_loss(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([-114.0371, -137.9123, -135.4440, -147.6318, -105.2794]) Cost: 90756.585938
Epoch    1/20 y_pred: tensor([3.6124, 3.4946, 3.8862, 4.0953, 2.5787]) Cost: 28448.974609
Epoch    2/20 y_pred: tensor([69.4798, 82.6634, 81.8919, 89.0417, 62.9647]) Cost: 8918.843750
Epoch    3/20 y_pred: tensor([106.3563, 126.9872, 125.5644, 136.6000,  96.7729]) Cost: 2797.183105
Epoch    4/20 y_pred: tensor([127.0019, 151.8027, 150.0150, 163.2261, 115.7010]) Cost: 878.367554
Epoch    5/20 y_pred: tensor([138.5603, 165.6961, 163.7039, 178.1330, 126.2984]) Cost: 276.919922
Epoch    6/20 y_pred: tensor([145.0312, 173.4747, 171.3677, 186.4787, 132.2318]) Cost: 88.397293
Epoch    7/20 y_pred: tensor([148.6538, 177.8298, 175.6584, 191.1512, 135.5538]) Cost: 29.304590
Epoch    8/20 y_pred: tensor([150.6817, 180.2682, 178.0604, 193.7670, 137.4139]) Cost: 10.781565
Epoch    9/20 y_pred: tensor([151.8168, 181.6336, 179.4052, 195.2315, 138.4556]) Cost: 4.974728
Epoch   10/20 y_pred: tensor(

# Lab-04-2 Loading Data

## Mini-Batch

### Dataset

In [46]:
from torch.utils.data import Dataset

In [47]:
x_train = torch.FloatTensor([[73, 80, 75], 
                             [93, 88, 93], 
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [48]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x = x_data
        self.y = y_data
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [49]:
train_data = CustomDataset(x_train, y_train)

In [50]:
train_data[0]

(tensor([73., 80., 75.]), tensor([152.]))

### DataLoader

In [51]:
from torch.utils.data import DataLoader

In [94]:
train_loader = DataLoader(dataset=train_data, batch_size=2, shuffle=True) 

In [95]:
train_loader

### Mini-Batch Gradient Descent

In [96]:
for batch_idx, samples in enumerate(train_loader):
    print(batch_idx)
    print(samples)

0
[tensor([[73., 80., 75.],
        [73., 66., 70.]]), tensor([[152.],
        [142.]])]
1
[tensor([[ 89.,  91.,  90.],
        [ 96.,  98., 100.]]), tensor([[180.],
        [196.]])]
2
[tensor([[93., 88., 93.]]), tensor([[185.]])]


In [97]:
n_epochs = 20
for epoch in range(n_epochs):
    for batch_idx, samples in enumerate(train_loader):
        x_train, y_train = samples
        
        output = model(x_train)
        
        loss = F.mse_loss(output, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        print('Epoch {:4d}/{} Batch: {}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, batch_idx+1, len(train_loader), loss.item()
        ))

Epoch    0/20 Batch: 1/3 Cost: 0.388091
Epoch    0/20 Batch: 2/3 Cost: 2.831610
Epoch    0/20 Batch: 3/3 Cost: 5.461881
Epoch    1/20 Batch: 1/3 Cost: 2.779845
Epoch    1/20 Batch: 2/3 Cost: 1.845090
Epoch    1/20 Batch: 3/3 Cost: 2.045185
Epoch    2/20 Batch: 1/3 Cost: 0.652663
Epoch    2/20 Batch: 2/3 Cost: 2.653633
Epoch    2/20 Batch: 3/3 Cost: 5.279631
Epoch    3/20 Batch: 1/3 Cost: 2.732191
Epoch    3/20 Batch: 2/3 Cost: 1.912549
Epoch    3/20 Batch: 3/3 Cost: 2.114500
Epoch    4/20 Batch: 1/3 Cost: 4.595750
Epoch    4/20 Batch: 2/3 Cost: 2.770309
Epoch    4/20 Batch: 3/3 Cost: 0.765945
Epoch    5/20 Batch: 1/3 Cost: 2.498620
Epoch    5/20 Batch: 2/3 Cost: 3.041684
Epoch    5/20 Batch: 3/3 Cost: 1.195188
Epoch    6/20 Batch: 1/3 Cost: 3.358598
Epoch    6/20 Batch: 2/3 Cost: 2.010214
Epoch    6/20 Batch: 3/3 Cost: 1.204045
Epoch    7/20 Batch: 1/3 Cost: 3.441093
Epoch    7/20 Batch: 2/3 Cost: 1.929599
Epoch    7/20 Batch: 3/3 Cost: 1.158934
Epoch    8/20 Batch: 1/3 Cost: 3.359736
